In [1]:
import torch
from torch import nn
# 데이터셋을 순회 가능한 객체(iterable)로 감쌈
from torch.utils.data import DataLoader
# CIFAR COCO와 같은 다양한 실제 비전 데이터에 대한 데이터셋
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [2]:
# 공개 데이터셋에서 학습 데이터 내려받기
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 공개 데이터셋에서 테스트 데이터 내려받기 
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
# dataset을 dataloader의 인자로 전달
# dataloader 객체의 각 요소는 64개의 feature이랑 정답label을 묶음batch로 반환함

batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
  print("Shape of X [N, C, H, W]:  ", X.shape)
  print("Shape of y:  ", y.shape, y.dtype)
  break

Shape of X [N, C, H, W]:   torch.Size([64, 1, 28, 28])
Shape of y:   torch.Size([64]) torch.int64


In [14]:
### 모델 만들기
# 신경망 모델은 nn.Module을 상속받는 클래스를 생성하여 정의함
# __init__ 함수에서 신경망의 계층layer을 정의하고
# forward 함수에서 신경망에 데이터를 어떻게 전달할지 지정함

# 학습에 사용할 cpu나 gpy 장치 얻기
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# 모델 정의하기
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork()
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [15]:
### 모델 매개변수 최적화하기

loss_fn = nn.CrossEntropyLoss() #손실함수
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) #옵티마이저

In [22]:
# 각 학습 단계 training loop에서 모델은 학습 데이터셋에 대한 예측을 수행하고
# 예측 오류를 역전파하여 모델의 매개변수를 조정함

def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    X, y = X, y

    # 예측 오류 계산
    pred = model(X)
    loss = loss_fn(pred, y)

    # 역전파
    optimizer.zero_grad()  # 역전파 실행하기 전에 변화도gradient를 0으로 만듦
    loss.backward()
    optimizer.step()  # 모델의 파라미터가 업데이트됨

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [23]:
# 테스트 데이터셋으로 모델의 성능을 확인함

def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()  # evaluation 과정에서 사용하지 않아야 하는 layer를 알아서 off
  test_loss, correct = 0, 0

  with torch.no_grad():  # 메모리 사용량 줄이려고
    for X, y in dataloader:
      X, y = X, y
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1)==y).type(torch.float).sum().item()
      
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [24]:
# 학습 단계는 여러 번의 반복 단계epochs를 거쳐서 수행됨
# 각 에폭에서 모델은 매개변수를 학습함

epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n---------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("DONE")

Epoch 1
---------------------------
loss: 2.304215  [    0/60000]
loss: 2.292180  [ 6400/60000]
loss: 2.273692  [12800/60000]
loss: 2.275316  [19200/60000]
loss: 2.240971  [25600/60000]
loss: 2.217540  [32000/60000]
loss: 2.238931  [38400/60000]
loss: 2.197631  [44800/60000]
loss: 2.185326  [51200/60000]
loss: 2.153460  [57600/60000]
Test Error: 
 Accuracy: 34.1%, Avg loss: 2.151400 

Epoch 2
---------------------------
loss: 2.159679  [    0/60000]
loss: 2.150655  [ 6400/60000]
loss: 2.087148  [12800/60000]
loss: 2.114189  [19200/60000]
loss: 2.047464  [25600/60000]
loss: 1.976957  [32000/60000]
loss: 2.027261  [38400/60000]
loss: 1.937827  [44800/60000]
loss: 1.930379  [51200/60000]
loss: 1.849954  [57600/60000]
Test Error: 
 Accuracy: 47.1%, Avg loss: 1.864992 

Epoch 3
---------------------------
loss: 1.891087  [    0/60000]
loss: 1.866256  [ 6400/60000]
loss: 1.746882  [12800/60000]
loss: 1.806772  [19200/60000]
loss: 1.680081  [25600/60000]
loss: 1.622669  [32000/60000]
loss: 1.

In [25]:
### 모델 저장하기
# 내부 상태 사전을 직렬화serialize함

torch.save(model.state_dict(), "model.pth")
print("Saved Pythorch Model State to model.pth")

Saved Pythorch Model State to model.pth


In [26]:
### 모델 불러오기
# 모델 구조를 다시 만들고 상태 사전을 모델에 불러옴

model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [27]:
# 이 모델로 예측할 수 있음

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
  pred = model(x)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(f'predicted: "{predicted}", Actual: "{actual}"')

predicted: "Ankle boot", Actual: "Ankle boot"


텐서